In [66]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
from scipy import stats
from sklearn.model_selection import train_test_split

In [34]:
# con = psycopg2.connect(
#     host = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com',
#     database = 'mid_term_project',
#     user = 'lhl_student',
#     password = 'lhl_student',
#     port = 5432
# )
# sql_flight_info = "SELECT fl_date, mkt_unique_carrier, op_unique_carrier, op_carrier_fl_num, origin,crs_dep_time, dep_time, dep_delay, dest, arr_time, arr_delay, crs_arr_time, crs_elapsed_time, actual_elapsed_time, distance, carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay FROM flights WHERE (origin = 'ORD' or origin = 'ATL' or origin = 'DFW' or origin = 'DEN' or origin = 'LAX' or origin = 'CLT' or origin = 'SEA' or origin = 'SFO' or origin = 'EWR' or origin = 'PHX' or origin = 'LGA' or origin = 'IAH' or origin = 'LAS' or origin = 'BOS' or origin = 'PHL' or origin = 'MCO' or origin = 'DTW' or origin = 'MSP' or origin = 'DCA') or (dest = 'ORD' or dest = 'ATL' or dest = 'DFW' or dest = 'DEN' or dest = 'LAX' or dest = 'CLT' or dest = 'SEA' or dest = 'SFO' or dest = 'EWR' or dest = 'PHX' or dest = 'LGA' or dest = 'IAH' or dest = 'LAS' or dest = 'BOS' or dest = 'PHL' or dest = 'MCO' or dest = 'DTW' or dest = 'MSP' or dest = 'DCA') ORDER BY RANDOM() LIMIT 1000000;"
# data_flight_info = pd.read_sql_query(sql_flight_info,con)
# con.close()

In [65]:
# data_flight_info.to_csv('training_data.csv', encoding='utf-8', index=False)

In [67]:
data_flight_info = pd.read_csv('/Users/craiglynch/Desktop/Lighthouse_Labs/Mid-term_Project/mid-term-project-I-master/SQL_data.csv', sep = ',')

In [68]:
# Creating a function which will remove extra leading 
# and tailing whitespace from the data.
# pass dataframe as a parameter here

def whitespace_remover(dataframe):
    
    # iterating over the columns
    for i in dataframe.columns:
          
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
              
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
        else:
              
            # if condn. is False then it will do nothing.
            pass

In [69]:
df_flight_info = data_flight_info.copy()
whitespace_remover(df_flight_info)
df_flight_info.shape

(1000000, 20)

In [70]:
# Creating year, month, date columns
# Create new columns
df_flight_info['year'] = pd.DatetimeIndex(df_flight_info['fl_date']).year
df_flight_info['month'] = pd.DatetimeIndex(df_flight_info['fl_date']).month
df_flight_info['day'] = pd.DatetimeIndex(df_flight_info['fl_date']).day
df_flight_info['weekday'] = ((pd.DatetimeIndex(df_flight_info['fl_date']).dayofweek) // 5 == 1).astype(float)
df_flight_info.isnull().sum()

fl_date                     0
mkt_unique_carrier          0
op_unique_carrier           0
op_carrier_fl_num           0
origin                      0
crs_dep_time                0
dep_time                16950
dep_delay               17240
dest                        0
arr_time                17987
arr_delay               20370
crs_arr_time                0
crs_elapsed_time            0
actual_elapsed_time     20209
distance                    0
carrier_delay          805310
weather_delay          805310
nas_delay              805310
security_delay         805310
late_aircraft_delay    805310
year                        0
month                       0
day                         0
weekday                     0
dtype: int64

In [71]:
# Filling all NaN delay info with 0
df_flight_info[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df_flight_info[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(0)

In [72]:
# Because dep_time, dep_delay, arr_time, arr_delay, and actual_elapsed_time have relatively small amounts of NaN they were dropped
df_flight_info = df_flight_info.dropna()

In [73]:
df_flight_info.isnull().sum()

fl_date                0
mkt_unique_carrier     0
op_unique_carrier      0
op_carrier_fl_num      0
origin                 0
crs_dep_time           0
dep_time               0
dep_delay              0
dest                   0
arr_time               0
arr_delay              0
crs_arr_time           0
crs_elapsed_time       0
actual_elapsed_time    0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
year                   0
month                  0
day                    0
weekday                0
dtype: int64

In [74]:
# 20,494 rows removed
df_flight_info.shape

(979360, 24)

In [75]:
# df_flight_info1 = df_flight_info[df_flight_info['carrier_delay'] <= 720]
# df_flight_info2 = df_flight_info1[df_flight_info1['weather_delay'] <= 720]
# df_flight_info3 = df_flight_info2[df_flight_info2['nas_delay'] <= 720]
# df_flight_info4 = df_flight_info3[df_flight_info3['security_delay'] <= 720]
# df_flight_info = df_flight_info4[df_flight_info4['late_aircraft_delay'] <= 720]
# #630 rows removed

In [76]:
df_flight_info.shape

(979360, 24)

In [77]:
#Only want top 19 busiest airports
airports = ['ORD','ATL','DFW','DEN','LAX','CLT','SEA','SFO','EWR','LGA','PHX','IAH','LAS','BOS','MCO','DTW','PHIL','DCA','MSP']
df_flight_info = df_flight_info[df_flight_info['origin'].isin(airports)]
df_flight_info.origin.value_counts()

ATL    59088
ORD    58718
DFW    43997
DEN    41751
CLT    37855
LAX    37724
SEA    29069
PHX    27111
SFO    27111
IAH    26870
DTW    26379
LGA    25182
LAS    25160
MSP    24546
EWR    23641
BOS    22596
DCA    21254
MCO    21118
Name: origin, dtype: int64

In [79]:
df_flight_info.isnull().sum()

fl_date                0
mkt_unique_carrier     0
op_unique_carrier      0
op_carrier_fl_num      0
origin                 0
crs_dep_time           0
dep_time               0
dep_delay              0
dest                   0
arr_time               0
arr_delay              0
crs_arr_time           0
crs_elapsed_time       0
actual_elapsed_time    0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
year                   0
month                  0
day                    0
weekday                0
dtype: int64

In [78]:
df_flight_info.to_csv('Classification_cleaned_data_with_outliers_01.csv', encoding='utf-8', index=False)

In [32]:
y = data_flight_info['arr_delay']
X = data_flight_info.drop(['arr_delay'],axis=1)